In [2]:
# ---------------------------------------------------
# 📦 Step 1: Import Required Libraries
# ---------------------------------------------------

# 'os' lets us work with environment variables and system settings.
import os

# Streamlit helps build interactive apps — but here, 
# we’ll just simulate in Jupyter (you can ignore Streamlit for now).
# If you later want to run it as a web app, you can use Streamlit.

# LangChain helps structure LLM apps with prompt templates, memory, and chains.
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory

# This is Gemini integration (instead of OpenAI)
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
# ---------------------------------------------------
# 🔑 Step 2: Set Up Your Gemini API Key
# ---------------------------------------------------

# Replace the below line with your actual Gemini API key
os.environ["GOOGLE_API_KEY"] = "YOUR_GEMINI_API_KEY_HERE"


In [17]:
# ---------------------------------------------------
# 🤖 Step 3: Initialize Gemini LLM
# ---------------------------------------------------

# We create a "llm" object that connects LangChain to Gemini.
# The 'model' parameter defines which Gemini model to use (latest is 'gemini-pro').
# The 'temperature' controls creativity: higher = more creative.

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.8)


In [ ]:
# ---------------------------------------------------
# 🧩 Step 4: Create Prompt Templates
# ---------------------------------------------------

# Each PromptTemplate defines *how* the model should be prompted.

# 1️⃣ First prompt asks for info about the celebrity
first_input_prompt = PromptTemplate(
    input_variables=['name'],        # variable passed into the prompt
    template="Tell me about celebrity {name}"  # actual prompt text
)

# 2️⃣ Second prompt asks for date of birth
second_input_prompt = PromptTemplate(
    input_variables=['person'], 
    template="{person} born?"
)

# 3️⃣ Third prompt asks for events around that date
third_input_prompt = PromptTemplate(
    input_variables=['dob'], 
    template="Mention 5 major world events that happened around {dob}"
)


In [19]:
# ---------------------------------------------------
# 🧠 Step 5: Create Memory Buffers
# ---------------------------------------------------

# ConversationBufferMemory allows the model to remember previous inputs/outputs.

person_memory = ConversationBufferMemory(input_key='name', memory_key='chat_history')
dob_memory = ConversationBufferMemory(input_key='person', memory_key='chat_history')
descr_memory = ConversationBufferMemory(input_key='dob', memory_key='description_history')


In [20]:
# ---------------------------------------------------
# 🔗 Step 6: Build Each Chain
# ---------------------------------------------------

# First chain: Get details about the celebrity
chain1 = LLMChain(
    llm=llm,
    prompt=first_input_prompt,
    verbose=True,
    output_key='person',  # output will be stored in variable 'person'
    memory=person_memory
)

# Second chain: Get the date of birth
chain2 = LLMChain(
    llm=llm,
    prompt=second_input_prompt,
    verbose=True,
    output_key='dob',
    memory=dob_memory
)

# Third chain: Get world events around that date
chain3 = LLMChain(
    llm=llm,
    prompt=third_input_prompt,
    verbose=True,
    output_key='description',
    memory=descr_memory
)


In [21]:
# ---------------------------------------------------
# 🔄 Step 7: Combine All Chains into One Sequential Flow
# ---------------------------------------------------

# SequentialChain runs multiple LLMChains in order, passing outputs between them.

parent_chain = SequentialChain(
    chains=[chain1, chain2, chain3],
    input_variables=['name'],  # initial input to the chain
    output_variables=['person', 'dob', 'description'],
    verbose=True
)


In [26]:
# ---------------------------------------------------
# 🧾 Step 8: Test the Chain
# ---------------------------------------------------

# Ask the user for input (in Jupyter, we’ll use input())
celebrity_name = input("Enter the name of a celebrity: ")

# Run the chain
output = parent_chain({'name': celebrity_name})


# ---------------------------------------------------
# 🪄 Step 9: Display the Results
# ---------------------------------------------------

print("\n🎤 Celebrity Information:")
print(output['person'])

print("\n🎂 Date of Birth:")
print(output['dob'])

print("\n🌍 Major Events Around That Time:")
print(output['description'])




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Tell me about celebrity varun dhawan

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
When was Varun Dhawan is a prominent Indian actor who primarily works in Hindi films. He is one of the most popular and commercially successful actors of his generation in Bollywood.

Here's a breakdown of his career and life:

1.  **Early Life and Background:**
    *   **Born:** April 24, 1987, in Mumbai, India.
    *   **Family:** He is the son of renowned film director David Dhawan and Karuna Dhawan. His elder brother, Rohit Dhawan, is also a film director.
    *   **Education:** Varun studied Business Management at Nottingham Trent University in the United Kingdom.
    *   **Before Acting:** Before making his acting debut, he worked as an assistant director to Karan Johar on the 2010 film *My Name Is Khan*.

2.  **Bollywood Debut and Rise to Stardom:**
    *   **Debut